In [2]:
import re, sys
from pathlib import Path
import numpy as np
import tifffile as tiff
import SimpleITK as sitk

# --- CONFIGURATION ---
input_dir   = r"F:\EVOS z stacks\Nov12\MCF-7\Z-Stack 2016-05-12 01.33.49\2"   # Folder containing all TIFF files
output_path = r"F:\EVOS z stacks\Nov12\MCF-7\Z-Stack 2016-05-12 01.33.49\2\mcf7_s.nii.gz"
gray_mode   = "blue"   # Use blue channel (best for DAPI)
recurse     = False    # Set True if TIFFs are inside subfolders
# ----------------------------------------------------------

p = Path(input_dir)
files = [f for f in (p.rglob("*") if recurse else p.glob("*"))
         if f.suffix.lower() in (".tif", ".tiff")]

if not files:
    raise SystemExit(f"No .tif/.tiff files found in {input_dir}")

# Sort by last 3 digits in filename (001, 002, ...)
def zindex(stem):
    m = re.search(r'(\d{3})(?=\D*$)', stem)
    return int(m.group(1)) if m else sys.maxsize

files = sorted(files, key=lambda f: (zindex(f.stem), f.name.lower()))
print(f"Found {len(files)} slices. Example: {[f.name for f in files[:5]]}")

# --- Load each TIFF and convert to grayscale ---
def to_gray(arr):
    if arr.ndim == 3 and arr.shape[-1] in (3,4):  # RGB/RGBA
        r, g, b = [arr[..., i].astype(np.float32) for i in (0,1,2)]
        if gray_mode == "blue":  return b
        if gray_mode == "green": return g
        if gray_mode == "red":   return r
        if gray_mode == "luma":  return 0.299*r + 0.587*g + 0.114*b
        if gray_mode == "max":   return np.max(arr[...,:3], axis=-1)
    elif arr.ndim == 3:          # multipage (Z,Y,X)
        return arr[0]
    elif arr.ndim == 2:
        return arr
    else:
        raise ValueError(f"Unsupported TIFF shape: {arr.shape}")

slices = [to_gray(tiff.imread(str(f))) for f in files]

# --- Validate shapes ---
ref_shape = slices[0].shape
for i, s in enumerate(slices):
    if s.shape != ref_shape:
        raise ValueError(f"Slice {files[i].name} has shape {s.shape}, expected {ref_shape}")

# --- Stack and save ---
volume = np.stack(slices, axis=0).astype(np.float32)
print("Stacked volume shape:", volume.shape)

img = sitk.GetImageFromArray(volume)
img.SetOrigin([0, 0, 0])
img.SetSpacing([1.0, 1.0, 1.0])  # Set to actual voxel size in mm if known (e.g., [0.00017, 0.00017, 0.001])
sitk.WriteImage(img, output_path)

print(f"✅ 3D NIfTI saved successfully at:\n{output_path}")


Found 132 slices. Example: ['Z-Stack GFP 001.tif', 'Z-Stack GFP 002.tif', 'Z-Stack GFP 003.tif', 'Z-Stack GFP 004.tif', 'Z-Stack GFP 005.tif']
Stacked volume shape: (132, 960, 1280)
✅ 3D NIfTI saved successfully at:
F:\EVOS z stacks\Nov12\MCF-7\Z-Stack 2016-05-12 01.33.49\2\mcf7_s.nii.gz
